In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
!pip install --upgrade treebank
!pip install nltk
import treebank
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.utils import to_categorical
#import libraries for model building
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding, SimpleRNN
# import 

     |████████████████████████████████| 2.0MB 5.1MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Loading Penn Treebank Dataset, and taking a part of it as training data**

In [ ]:
len(treebank.penn['train']) #characters
treebank.penn['train'][0:10]
type(treebank.penn['train'])

str

In [ ]:
data = treebank.penn['train']
data_train = data[:100000]

**Data Preprocessing**

In [ ]:
# stop_words = set(stopwords.words('english'))
stop_words = set()
stop_words.add('unk')
stop_words.add('>')
stop_words.add('<')
word_tokens = word_tokenize(data_train)
filtered_sentence = [w for w in word_tokens if not w in stop_words]

print(word_tokens[10:50])
print(filtered_sentence[10:50])


['ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack-food', 'ssangyong', 'swapo', 'wachter', 'pierre', '<', 'unk', '>', 'N', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', 'N', 'mr.', '<', 'unk', '>', 'is', 'chairman', 'of', '<', 'unk']
['ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack-food', 'ssangyong', 'swapo', 'wachter', 'pierre', 'N', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', 'N', 'mr.', 'is', 'chairman', 'of', 'n.v.', 'the', 'dutch', 'publishing', 'group', 'rudolph', 'N', 'years']


In [ ]:
data_f = " "
data_final = data_f.join(filtered_sentence)
# len(filtered_sentence)
# print(data_final)
# print(type(data_final))

**Tokenzing : Converting sentences into tokens, and giving numbers to them**

In [ ]:
tokenizer=Tokenizer(num_words=None,filters='#$%&"()*+-<=>@[\\]^_`{|}~\t\n',lower = True, split = ' ')
tokenizer.fit_on_texts([data_final])

encoded=tokenizer.texts_to_sequences([data_final])[0]
vocab_size=len(tokenizer.word_index)+1

**Creating input and output featues**

In [ ]:
#create seperate lists of 2 sequences since we will use 1 of them as input and one as output
sequences=[]
for i in range(len(encoded)-2):
    sequences.append(encoded[i:i+2])
# sequences[:50]

In [ ]:
#create dependent and independent variable

x=[]
y=[]
# x = sequences[:,0]
# y = sequences[:,1]
for i in range(len(sequences)):
    x.append(sequences[i][0])
    y.append(sequences[i][1])

In [ ]:
#convert list to an array
x=np.array(x)
y=np.array(y)
# One hot encoding
y=to_categorical(y,vocab_size)
# y.shape

In [ ]:
# x_test = x[:200000]
# y_test = y[:200000]

(17268, 3107)

**Define Single Layer Simple RNN model**

In [ ]:
#start building model
model=Sequential()
#add Embedding layer
model.add(Embedding(input_dim=vocab_size,output_dim=10,input_length=1))
#add LSTM layer
model.add(SimpleRNN(60))
#output layer
model.add(Dense(vocab_size,activation='softmax'))

In [ ]:
#compile 
model.compile('adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#training model
model.fit(x,y,batch_size=1,epochs=20,validation_split=0.2)

Epoch 1/20
12988/12988 [==============================] - 40s 3ms/step - loss: 7.0136 - accuracy: 0.0602 - val_loss: 7.6543 - val_accuracy: 0.0625
Epoch 2/20
12988/12988 [==============================] - 40s 3ms/step - loss: 6.1915 - accuracy: 0.0933 - val_loss: 8.6021 - val_accuracy: 0.0779
Epoch 3/20
12988/12988 [==============================] - 40s 3ms/step - loss: 5.9180 - accuracy: 0.1244 - val_loss: 9.2845 - val_accuracy: 0.0767
Epoch 4/20
12988/12988 [==============================] - 40s 3ms/step - loss: 5.8370 - accuracy: 0.1490 - val_loss: 9.3259 - val_accuracy: 0.0807
Epoch 5/20
12988/12988 [==============================] - 41s 3ms/step - loss: 5.5964 - accuracy: 0.1755 - val_loss: 9.5196 - val_accuracy: 0.0828
Epoch 6/20
12988/12988 [==============================] - 40s 3ms/step - loss: 5.5197 - accuracy: 0.1899 - val_loss: 9.6156 - val_accuracy: 0.0739
Epoch 7/20
12988/12988 [==============================] - 41s 3ms/step - loss: 5.4291 - accuracy: 0.1980 - val_loss: 9

**Predict function, which predicts `n_words` from the given `text`.**

In [ ]:
#testing function
def predict_next_word(model,tokenizer,text,n_words):
    result=''
    out_word=[]
    
    for i in range(n_words):
#         input_data=text.split()
        encoded_data=tokenizer.texts_to_sequences([text])[0]
        encoded_data=np.array(encoded_data)
        encoded_data=encoded_data.reshape(1,1)
        

        output=np.argmax(model.predict(encoded_data))
        
        
        for index,word in tokenizer.word_index.items():
            if word==output:
                out_word.append(index)
                text=index
    result=' '.join(out_word)
    return result

**Testing model**

In [ ]:
#testing 
predict_next_word(model,tokenizer,"harvard",10)

'funds were at the u.s. and other n n n'